In [1]:
### DO NOT RUN THIS.  It builds the BkAvgAst... tables, which are supplied in the processing tables zip file.
### 
### NOTE:  Major change due to conversion of some bank holding companies back to bank structures.
###        (This removes the Parent (NAMEHCR) and ParRSSD (RSSDHCR) fields from the institutions2.zip listing.)
###        

import numpy as np
import pandas as pd
from datetime import datetime
#import pandas_datareader.data as web
import warnings
warnings.filterwarnings('ignore')

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.chdir('C:/BankRisk')


In [2]:
rk = pd.read_csv("BkAvgAstPR_FailDate_202012.csv", index_col=0)
rk['xAsOf'] = pd.to_datetime(rk['xAsOf'])
rk['nAsOf'] = pd.to_datetime(rk['nAsOf'])
rk['FailDate'] = pd.to_datetime(rk['FailDate'])
rk.head()

Rank  IDRSSD      xAsOf                    xName xState    xCert  \
0     1  852218 2020-12-31  JPMORGAN CHASE BANK, NA     OH    628.0   
1     2  480228 2020-12-31      BANK OF AMERICA, NA     NC   3510.0   
2     3  476810 2020-12-31           CITIBANK, N.A.     SD   7213.0   
3     4  451965 2020-12-31     WELLS FARGO BANK, NA     SD   3511.0   
4     5  484422 2009-12-31        WACHOVIA BANK, NA     NC  33869.0   

       nAsOf      aeTotAst      aaTotAst       aeTotLn        aeDDep  \
0 2001-03-31  1.593288e+09  1.567517e+09  5.652142e+08  7.671177e+08   
1 2001-03-31  1.341065e+09  1.339664e+09  6.952804e+08  8.832854e+08   
2 2001-03-31  1.111830e+09  1.108563e+09  5.233710e+08  3.160202e+08   
3 2001-03-31  9.768041e+08  9.652788e+08  5.772582e+08  6.953574e+08   
4 2001-03-31  4.416464e+08  4.372459e+08  2.546369e+08  2.704156e+08   

         aeFDep      aeTotDep   FailDate    pr_eTA   pr_eTAcs    pr_eTL  \
0  2.613865e+08  1.028504e+09        NaT  9.324842   9.324842  5.825125   
1  1.040564e+08  9.873418e+08 2009-01-16  7.848688  17.173530  7.165594   
2  4.547897e+08  7.708099e+08 2008-11-23  6.507073  23.680603  5.393887   
3  6.515955e+07  7.605170e+08        NaT  5.716821  29.397424  5.949251   
4  2.469325e+07  2.951088e+08        NaT  2.584770  31.982194  2.624301   

    pr_eTLcs    pr_eDD   pr_eDDcs  
0   5.825125  7.905952   7.905952  
1  12.990719  9.103182  17.009134  
2  18.384606  3.256919  20.266053  
3  24.333857  7.166387  27.432440  
4  26.958158  2.786916  30.219356

In [3]:
len(rk) # 10798

10798

In [4]:
bhc = pd.read_csv("INSTITUTIONS2_202104.csv", encoding='latin-1') # Warning: This is latest available file as of 2021-April.
bhc.head()

STNAME  STMULT  CERT  DOCKET  ACTIVE              ADDRESS      ASSET  \
0  Connecticut     NaN     4       0       0       22 Main Street        NaN   
1  Connecticut     NaN     6       0       0  81 West Main Street        NaN   
2        Maine     NaN     8       0       0      One City Center  1,699,404   
3        Maine     NaN     9    9781       0       66 Main Street    539,169   
4        Maine     NaN    10       0       0  8 Washington Street        NaN   

  BKCLASS  CHANGEC1  CHANGEC2  ...                                   CBSA  \
0      NM       223         0  ...  Hartford-East Hartford-Middletown, CT   
1      NM       223       999  ...                  New Haven-Milford, CT   
2      SM       223         0  ...            Portland-South Portland, ME   
3      SM       223         0  ...                                    NaN   
4      SM       223         0  ...            Portland-South Portland, ME   

   CBSA_NO                        CBSA_METRO_NAME  CBSA_METRO CBSA_METRO_FLG  \
0    25540  Hartford-East Hartford-Middletown, CT       25540              1   
1    35300                  New Haven-Milford, CT       35300              1   
2    38860            Portland-South Portland, ME       38860              1   
3        0                                    NaN           0              0   
4    38860            Portland-South Portland, ME       38860              1   

  CBSA_MICRO_FLG CBSA_DIV  CBSA_DIV_NO  CBSA_DIV_FLG  CB  
0              0      NaN            0             0 NaN  
1              0      NaN            0             0 NaN  
2              0      NaN            0             0 NaN  
3              0      NaN            0             0 NaN  
4              0      NaN            0             0 NaN  

[5 rows x 134 columns]

In [5]:
bhc.replace(to_replace='orporated', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='Irrevocable', value='Irrev', inplace=True, regex=True)
bhc.replace(to_replace='Employee Stock Ownership ', value='ESO', inplace=True, regex=True)
bhc.replace(to_replace='orporation', value='orp', inplace=True, regex=True)
bhc.replace(to_replace='Financial', value='Fin', inplace=True, regex=True)
bhc.replace(to_replace='Services', value='Serv', inplace=True, regex=True)
bhc.replace(to_replace='Holding Company', value='HC', inplace=True, regex=True)
#bhc.replace(to_replace=', Inc.', value=' Inc', inplace=True, regex=True)
bhc.replace(to_replace='Inc.', value='Inc', inplace=True, regex=True)
bhc.replace(to_replace='Corp.', value='Corp', inplace=True, regex=True)
bhc.replace(to_replace=', The', value=' The', inplace=True, regex=True)
bhc.replace(to_replace='orpThe', value='orp', inplace=True, regex=True)

In [6]:
len(bhc) # 27627

27627

In [7]:
bhc.head(2)

STNAME  STMULT  CERT  DOCKET  ACTIVE              ADDRESS ASSET  \
0  Connecticut     NaN     4       0       0       22 Main Street   NaN   
1  Connecticut     NaN     6       0       0  81 West Main Street   NaN   

  BKCLASS  CHANGEC1  CHANGEC2  ...                                   CBSA  \
0      NM       223         0  ...  Hartford-East Hartford-Middletown, CT   
1      NM       223       999  ...                  New Haven-Milford, CT   

   CBSA_NO                        CBSA_METRO_NAME  CBSA_METRO CBSA_METRO_FLG  \
0    25540  Hartford-East Hartford-Middletown, CT       25540              1   
1    35300                  New Haven-Milford, CT       35300              1   

  CBSA_MICRO_FLG CBSA_DIV  CBSA_DIV_NO  CBSA_DIV_FLG  CB  
0              0      NaN            0             0 NaN  
1              0      NaN            0             0 NaN  

[2 rows x 134 columns]

In [8]:
bhc2=bhc[['CERT','FED_RSSD','NAME','STALP','BKCLASS','OFFICES','NAMEHCR','RSSDHCR','STALPHCR','ULTCERT']]
#bhc2.columns = [['Cert','IDRSSD','Name','ST','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']]
bhc2.rename(index=str, columns={"CERT":"Cert", "FED_RSSD":"IDRSSD", "NAME":"Name", "STALP":"ST", "BKCLASS":"BkClass", \
            "OFFICES":"Offices", "NAMEHCR":"Parent", "RSSDHCR":"ParRSSD", "STALPHCR":"ParST", "ULTCERT":"UltCert"}, inplace=True)
len(bhc2) # 27627

27627

In [9]:
bhc2.head(2)

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   

  Parent  ParRSSD ParST  UltCert  
0    NaN      NaN   NaN     3510  
1    NaN      NaN   NaN     3510

In [10]:
bhc2.columns

Index(['Cert', 'IDRSSD', 'Name', 'ST', 'BkClass', 'Offices', 'Parent',
       'ParRSSD', 'ParST', 'UltCert'],
      dtype='object')

In [11]:
bhc2[bhc2['Parent']>' '][0:10]

Cert  IDRSSD                                 Name  ST BkClass  Offices  \
5     14   35301  State Street Bank and Trust Company  MA      SM      3.0   
15    35  749635                           AuburnBank  AL      SM      8.0   
19    39  430036            Robertson Banking Company  AL      NM      7.0   
21    41  432432                   Phenix-Girard Bank  AL      NM      4.0   
25    46  113937            Merchants Bank of Alabama  AL      NM      5.0   
29    50  470032                Community Spirit Bank  AL      NM      6.0   
30    51  382537                   The Bank of Vernon  AL      NM      4.0   
31    52  293437        The Citizens Bank of Winfield  AL      NM      1.0   
33    54  672537         The Exchange Bank of Alabama  AL      NM      5.0   
37    58  994435                          United Bank  AL      NM     22.0   

                            Parent    ParRSSD ParST  UltCert  
5                State Street Corp  1111435.0    MA       14  
15       Auburn National Bcorp Inc  1129533.0    AL       35  
19                         Rbc Inc  2526755.0    AL       39  
21    Phenix-Girard Bancshares Inc  2013246.0    AL       41  
25         Merchants Finl Serv Inc  1082955.0    AL       46  
29  Independent Bancshares Employe  3587294.0    AL       50  
30     First Vernon Bancshares Inc  1139998.0    AL       51  
31  Citizens Bcorp Of Winfield Inc  1833616.0    AL       52  
33                    Gadsden Corp  1079067.0    AL       54  
37     United Bcorp Of Alabama Inc  1083895.0    AL       58

In [12]:
#bhc2[(bhc2['ParRSSD']>0)&pd.isnull(bhc2['Parent'])]  ## should be null list

In [13]:
#bhc2[(bhc2['UltCert']>0)&pd.isnull(bhc2['Parent'])].head()

In [14]:
## extract ticker info from prior listing: (rkhn was expanded output from br001f_NewBHCtickers code)
rkh = pd.read_csv("BkAvgAstPR_FailDate_BHC2_ult202003.csv", index_col=0)

#len(rkh) # 10775
len(rkh) # 10792

10792

In [15]:
len(rkh[rkh['Ticker']>'']) # 3446

3446

In [16]:
#uex = hs3[['Date','UE','UE_'+st]].ix[hs3['UE'].idxmax()]
rkh2 = rkh[rkh['ParRSSD']>0][['Rank','ParRSSD','ParST','Ticker']]
len(rkh2) # 9150

9150

In [17]:
rkh2.tail(2)

Rank    ParRSSD ParST Ticker
10788  10789  1094314.0    MO  CBCYB
10791  10792  3447567.0    TX    NaN

In [18]:
rkh3 = rkh2.groupby(['ParRSSD','Ticker']).min()
rkh3.sort_values(['Rank'],inplace=True)
len(rkh3) # 401

401

In [19]:
rkh3.head()

Rank ParST
ParRSSD   Ticker            
1039502.0 JPM        1    NY
1073757.0 BAC        2    NC
1951350.0 C          3    NY
1120754.0 WFC        4    CA
1119794.0 USB        6    MN

In [20]:
rkh4 = rkh3.reset_index(drop=False)
rkh4.head()

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN

In [21]:
rkh5=rkh4[['ParRSSD','Ticker']].groupby('Ticker').count()
rkh5.head()

ParRSSD
Ticker         
ABCB          1
ABTX          1
ABYB          1
ACBI          1
ACNB          1

In [22]:
rkh5[rkh5['ParRSSD']>1] # finds tickers with more than one ParRSSD code:

Empty DataFrame
Columns: [ParRSSD]
Index: []

In [23]:
rkh[rkh['Ticker']=="BHWB"].iloc[:,0:12] # check on Ticker that was misidentified in earlier runs 

Rank  IDRSSD       xAsOf                                xName xState  \
1712  1713   27548  2020-03-31                       BLACKHAWK BANK     WI   
5322  5323  231336  2019-06-30  FIRST NATIONAL BANK OF MCHENRY, THE     IL   
7795  7796  388931  2003-09-30                        FIRST BANK BC     IL   

        xCert       nAsOf FailDate BkClass  Offices               Parent  \
1712  14078.0  2001-03-31      NaN      NM     12.0  Blackhawk Bcorp Inc   
5322  20081.0  2001-03-31      NaN       N      3.0  Blackhawk Bcorp Inc   
7795  16551.0  2001-03-31      NaN      NM      4.0  Blackhawk Bcorp Inc   

        ParRSSD  
1712  1491913.0  
5322  1491913.0  
7795  1491913.0

In [24]:
bhc3 = bhc2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
bhc3.head()

Cert  IDRSSD                                    Name  ST BkClass  Offices  \
0     4  573401  The Southington Bank and Trust Company  CT      NM      4.0   
1     6  148304              Colonial Bank of Waterbury  CT      NM     34.0   
2     8  422406                     Fleet Bank of Maine  ME      SM     55.0   
3     9  563907                     Union Trust Company  ME      SM     13.0   
4    10  112109               Northeast Bank of Sanford  ME      SM      5.0   

  Parent  ParRSSD ParST  UltCert Ticker  
0    NaN      NaN   NaN     3510    NaN  
1    NaN      NaN   NaN     3510    NaN  
2    NaN      NaN   NaN     3510    NaN  
3    NaN      NaN   NaN     4255    NaN  
4    NaN      NaN   NaN     3510    NaN

In [25]:
bhc3[bhc3['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert Ticker  
2233   4239.0  Bank Of America Corp  1073757.0    NC     3510    BAC

In [26]:
bhc2[bhc2['Cert']==3510]

Cert  IDRSSD                                   Name  ST BkClass  \
2233  3510  480228  Bank of America, National Association  NC       N   

      Offices                Parent    ParRSSD ParST  UltCert  
2233   4239.0  Bank Of America Corp  1073757.0    NC     3510

In [27]:
rk2 = rk.merge(bhc2[['Cert','BkClass','Offices','Parent','ParRSSD','ParST','UltCert']],how='left',left_on='xCert',\
              right_on='Cert')
rk2.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2020-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2020-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...   pr_eTLcs    pr_eDD  \
0  1.593288e+09  1.567517e+09  5.652142e+08  ...   5.825125  7.905952   
1  1.341065e+09  1.339664e+09  6.952804e+08  ...  12.990719  9.103182   

    pr_eDDcs    Cert  BkClass  Offices                Parent    ParRSSD  \
0   7.905952   628.0        N   4987.0     Jpmorgan Chase&Co  1039502.0   
1  17.009134  3510.0        N   4239.0  Bank Of America Corp  1073757.0   

   ParST  UltCert  
0     NY    628.0  
1     NC   3510.0  

[2 rows x 27 columns]

In [28]:
rkh4.head(7)

ParRSSD Ticker  Rank ParST
0  1039502.0    JPM     1    NY
1  1073757.0    BAC     2    NC
2  1951350.0      C     3    NY
3  1120754.0    WFC     4    CA
4  1119794.0    USB     6    MN
5  1069778.0    PNC     7    PA
6  3587146.0     BK    10    NY

In [29]:
rk3 = rk2.merge(rkh4[['ParRSSD','Ticker']],how='left',on='ParRSSD')
rk3.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2020-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2020-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

       aeTotAst      aaTotAst       aeTotLn  ...    pr_eDD   pr_eDDcs    Cert  \
0  1.593288e+09  1.567517e+09  5.652142e+08  ...  7.905952   7.905952   628.0   
1  1.341065e+09  1.339664e+09  6.952804e+08  ...  9.103182  17.009134  3510.0   

  BkClass  Offices                Parent    ParRSSD  ParST  UltCert  Ticker  
0       N   4987.0     Jpmorgan Chase&Co  1039502.0     NY    628.0     JPM  
1       N   4239.0  Bank Of America Corp  1073757.0     NC   3510.0     BAC  

[2 rows x 28 columns]

In [30]:
rk3c = list(rk3.columns)
rk3c2 = rk3c[0:7] + ['FailDate'] + rk3c[-7:] + rk3c[7:13] + rk3c[14:20]
rk3c, rk3c2

(['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'FailDate',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs',
  'Cert',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker'],
 ['Rank',
  'IDRSSD',
  'xAsOf',
  'xName',
  'xState',
  'xCert',
  'nAsOf',
  'FailDate',
  'BkClass',
  'Offices',
  'Parent',
  'ParRSSD',
  'ParST',
  'UltCert',
  'Ticker',
  'aeTotAst',
  'aaTotAst',
  'aeTotLn',
  'aeDDep',
  'aeFDep',
  'aeTotDep',
  'pr_eTA',
  'pr_eTAcs',
  'pr_eTL',
  'pr_eTLcs',
  'pr_eDD',
  'pr_eDDcs'])

In [31]:
rk4 = rk3[rk3c2]
rk4.head(2)

Rank  IDRSSD      xAsOf                    xName xState   xCert      nAsOf  \
0     1  852218 2020-12-31  JPMORGAN CHASE BANK, NA     OH   628.0 2001-03-31   
1     2  480228 2020-12-31      BANK OF AMERICA, NA     NC  3510.0 2001-03-31   

    FailDate BkClass  Offices  ...       aeTotLn        aeDDep        aeFDep  \
0        NaT       N   4987.0  ...  5.652142e+08  7.671177e+08  2.613865e+08   
1 2009-01-16       N   4239.0  ...  6.952804e+08  8.832854e+08  1.040564e+08   

       aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD   pr_eDDcs  
0  1.028504e+09  9.324842   9.324842  5.825125   5.825125  7.905952   7.905952  
1  9.873418e+08  7.848688  17.173530  7.165594  12.990719  9.103182  17.009134  

[2 rows x 27 columns]

In [32]:
## New code (May 2019) to add holding company values for Parent, ParRSSD, Ticker for Zions Bancorp (since this data)
## was recently removed due to legal changes to terminate Fed CCAR oversight
## If other companies do the same, their data will be added back here
## (This is essential for the historical CCAR 31-bank group to compute properly)

In [33]:
rk4.loc[rk4.xCert.isin([2270]),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']]

IDRSSD                     xName   xCert Ticker Parent  ParRSSD
73  276579  ZIONS BANCORPORATION, NA  2270.0    NaN    NaN      NaN

In [34]:
rk4.loc[rk4.xCert == 2270, 'Parent'] = 'ZION'
rk4.loc[rk4.xCert == 2270, 'Ticker'] = 'ZION'
rk4.loc[rk4.xCert == 2270, 'ParRSSD'] = 1027004.0

In [35]:
rk4.loc[rk4.xCert.isin([2270]),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']]

IDRSSD                     xName   xCert Ticker Parent    ParRSSD
73  276579  ZIONS BANCORPORATION, NA  2270.0   ZION   ZION  1027004.0

In [36]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC_nohist202012.csv")

In [37]:
len(rk4) # 10798

10798

In [38]:
## find bhcs with totast above 700MM and no ticker:
rkm = rk4[(rk4['ParRSSD']>0)&(rk4['aeTotAst']>700000)&pd.isnull(rk4['Ticker'])]
len(rkm) # 255

255

In [39]:
rkm.head()

Rank   IDRSSD      xAsOf  \
33     34   619877 2020-12-31   
75     76  2735146 2020-12-31   
115   116  2590037 2020-12-31   
124   125   595270 2020-12-31   
141   142   915878 2020-12-31   

                                                 xName xState    xCert  \
33                           USAA FEDERAL SAVINGS BANK     TX  32188.0   
75                                           TIAA, FSB     FL  34775.0   
115                               STATE FARM BANK, FSB     IL  34617.0   
124                                      MIDFIRST BANK     OK   4063.0   
141  THIRD FEDERAL SAVINGS & LOAN ASSOCIATION OF CL...     OH  30012.0   

         nAsOf FailDate BkClass  Offices  ...       aeTotLn        aeDDep  \
33  2012-03-31      NaT      SA      1.0  ...  4.430776e+07  6.727625e+07   
75  2012-03-31      NaT      SA     14.0  ...  2.395874e+07  1.983912e+07   
115 2012-03-31      NaT       N      2.0  ...  1.000623e+07  1.028637e+07   
124 2012-03-31      NaT      SA     81.0  ...  1.128825e+07  8.281213e+06   
141 2012-03-31      NaT      SA     37.0  ...  1.174344e+07  8.738349e+06   

    aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD  \
33     0.0  6.727625e+07  0.445102  55.220660  0.456638  48.121056  0.693352   
75     0.0  1.983912e+07  0.166298  66.784790  0.246920  61.895067  0.204463   
115    0.0  1.028637e+07  0.091854  71.641285  0.103125  67.170544  0.106012   
124    0.0  8.281213e+06  0.085601  72.432635  0.116337  68.027762  0.085347   
141    0.0  8.738349e+06  0.075528  73.796026  0.121028  69.602302  0.090058   

      pr_eDDcs  
33   54.849619  
75   67.815454  
115  73.735960  
124  74.651769  
141  76.284792  

[5 rows x 27 columns]

In [40]:
rkm.to_csv('rk_missingtickers202012.csv')

In [41]:
##############################################################################

In [42]:
## start up here after filling in missing tickers

In [43]:
## filled in many missing tickers

In [44]:
rkm2 = pd.read_csv('rk_missingtickers202012a.csv',index_col=0)
rkm2.head()

Rank   IDRSSD       xAsOf  \
33     34   619877  12/31/2020   
75     76  2735146  12/31/2020   
115   116  2590037  12/31/2020   
124   125   595270  12/31/2020   
141   142   915878  12/31/2020   

                                                 xName xState  xCert  \
33                           USAA FEDERAL SAVINGS BANK     TX  32188   
75                                           TIAA, FSB     FL  34775   
115                               STATE FARM BANK, FSB     IL  34617   
124                                      MIDFIRST BANK     OK   4063   
141  THIRD FEDERAL SAVINGS & LOAN ASSOCIATION OF CL...     OH  30012   

         nAsOf FailDate BkClass  Offices  ...      aeTotLn        aeDDep  \
33   3/31/2012      NaN      SA        1  ...  44307758.47  6.727625e+07   
75   3/31/2012      NaN      SA       14  ...  23958744.06  1.983912e+07   
115  3/31/2012      NaN       N        2  ...  10006232.58  1.028637e+07   
124  3/31/2012      NaN      SA       81  ...  11288253.50  8.281213e+06   
141  3/31/2012      NaN      SA       37  ...  11743440.81  8.738349e+06   

    aeFDep      aeTotDep    pr_eTA   pr_eTAcs    pr_eTL   pr_eTLcs    pr_eDD  \
33     0.0  6.727625e+07  0.445102  55.220659  0.456638  48.121055  0.693352   
75     0.0  1.983912e+07  0.166298  66.784788  0.246920  61.895066  0.204463   
115    0.0  1.028637e+07  0.091854  71.641283  0.103125  67.170544  0.106012   
124    0.0  8.281213e+06  0.085601  72.432632  0.116337  68.027761  0.085347   
141    0.0  8.738349e+06  0.075528  73.796024  0.121028  69.602302  0.090058   

      pr_eDDcs  
33   54.849618  
75   67.815453  
115  73.735959  
124  74.651768  
141  76.284792  

[5 rows x 27 columns]

In [45]:
rkm3 = rkm2[rkm2['Ticker']>' '][['ParRSSD','Ticker']]
rkm3.head()

ParRSSD Ticker
200  1427239    EBC
334  5530258   CARE
391  1250101    FAF
692  2433312    AMP

In [46]:
rkm3p = set(rkm3['ParRSSD'])
len(rkm3), len(rkm3p) # 4, 4

(4, 4)

In [47]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','Ticker']]

xName                   Parent Ticker
200                  EASTERN BANK   Eastern Bankshares Inc    NaN
334           CARTER BANK & TRUST    Carter Bankshares Inc    NaN
391  FIRST AMERICAN TRUST, F.S.B.  First American Fin Corp    NaN
692          AMERIPRISE BANK, FSB       Ameriprise Fin Inc    NaN

In [48]:
rk4c = rk4.copy()

In [49]:
tdct = dict(zip(rkm3['ParRSSD'],rkm3['Ticker']))
tdct

{1427239: 'EBC', 5530258: 'CARE', 1250101: 'FAF', 2433312: 'AMP'}

In [50]:
## update missing Tickers if it exists:
for i in np.arange(len(rk4)):
  rs = rk4.loc[i]['ParRSSD']
  try:
    rk4.loc[i,'Ticker'] = tdct[rs]
  except:
    pass

In [51]:
rk4[rk4['ParRSSD'].isin(rkm3p)][['xName','Parent','ParRSSD','Ticker']]

xName                   Parent    ParRSSD Ticker
200                  EASTERN BANK   Eastern Bankshares Inc  1427239.0    EBC
334           CARTER BANK & TRUST    Carter Bankshares Inc  5530258.0   CARE
391  FIRST AMERICAN TRUST, F.S.B.  First American Fin Corp  1250101.0    FAF
692          AMERIPRISE BANK, FSB       Ameriprise Fin Inc  2433312.0    AMP

In [52]:
len(rk4[rk4['Ticker']>' ']),len(rk4c[rk4c['Ticker']>' ']) # 498, 494

(498, 494)

In [53]:
rk4.to_csv("BkAvgAstPR_FailDate_BHC2_nohist202012.csv")

In [54]:
## process ultcert version:

In [55]:
rk5 = rk4.copy()

In [56]:
ult = set(rk5['UltCert'])
len(ult) # 5353

5353

In [57]:
bhcc = rk5[rk5['xCert'].isin(ult)][['xCert','Parent','Ticker','ParRSSD','ParST']]
bhcc.head()

xCert                Parent Ticker    ParRSSD ParST
0   628.0     Jpmorgan Chase&Co    JPM  1039502.0    NY
1  3510.0  Bank Of America Corp    BAC  1073757.0    NC
2  7213.0         Citigroup Inc      C  1951350.0    NY
3  3511.0   Wells Fargo&Company    WFC  1120754.0    CA
5  6548.0             U S Bcorp    USB  1119794.0    MN

In [58]:
bhcc.dropna(subset=['Parent'],inplace=True)
len(bhcc) # 4100

4100

In [59]:
len(bhcc[bhcc['Ticker']>'']) # 495

495

In [60]:
bhcc.head(10)

xCert                   Parent Ticker    ParRSSD ParST
0     628.0        Jpmorgan Chase&Co    JPM  1039502.0    NY
1    3510.0     Bank Of America Corp    BAC  1073757.0    NC
2    7213.0            Citigroup Inc      C  1951350.0    NY
3    3511.0      Wells Fargo&Company    WFC  1120754.0    CA
5    6548.0                U S Bcorp    USB  1119794.0    MN
6    6384.0  Pnc Finl Serv Group Inc    PNC  1069778.0    PA
9     639.0   Bank Of Ny Mellon Corp     BK  3587146.0    NY
11     14.0        State Street Corp    STT  1111435.0    MA
13   9846.0          Truist Fin Corp    BBT  1074156.0    NC
14  57890.0        Hsbc Holdings Plc   HSBC  1857108.0     0

In [61]:
bhcc.groupby('xCert').filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

xCert                          Parent Ticker    ParRSSD ParST
36     16571.0                Bank Of Montreal    BMO  1231333.0     0
167     4999.0                   Cadence Bcorp   CADE  4882802.0    TX
256    58564.0               Flushing Fin Corp   FFIC  2393274.0    NY
436    26342.0            Royal Bank Of Canada     RY  1232497.0     0
661    35525.0     Atlantic Capital Bancshares   ACBI  3555686.0    GA
705     4999.0                   Cadence Bcorp   CADE  4882802.0    TX
1032    8888.0                    Ccb Fin Corp    NaN  1053496.0    MO
1053   35525.0     Atlantic Capital Bancshares   ACBI  3555686.0    GA
1210   57197.0                     Bnccorp Inc   BNCC  1248162.0    ND
1525    3116.0  Lone Star State Bancshares Inc    NaN  3641767.0    TX
1756   10973.0          Alliance Finl Serv Inc    NaN  1127146.0    MN
1795      89.0       White River Bancshares Co    NaN  3306589.0    AR
1841   57497.0            Pathfinder Bcorp Inc    NaN  2596776.0    NY
1870   16571.0                Bank Of Montreal    BMO  1231333.0     0
2270   58564.0               Flushing Fin Corp   FFIC  2393274.0    NY
2526   13198.0      Cornerstone Holding Co Inc    NaN  3435283.0    ND
2678   20776.0           Border Bancshares Inc    NaN  2238544.0    MN
2745   26342.0            Royal Bank Of Canada     RY  1232497.0     0
3367   35274.0       Blue Ridge Bankshares Inc    NaN  1426960.0    VA
4102   11310.0                   Fgh Bcorp Inc    NaN  2527024.0    IL
4325    8252.0        Jamesmark Bancshares Inc    NaN  2816803.0    MO
4483   13198.0      Cornerstone Holding Co Inc    NaN  3435283.0    ND
4503   17636.0     First Belleville Bancshares    NaN  1060917.0    KS
5815    1756.0     Cross County Bancshares Inc    NaN  1131611.0    AR
6093    3116.0  Lone Star State Bancshares Inc    NaN  3641767.0    TX
6767   17399.0                  First Fin Corp    NaN  1067514.0    KS
7212      89.0       White River Bancshares Co    NaN  3306589.0    AR
7220   10973.0          Alliance Finl Serv Inc    NaN  1127146.0    MN
7416   13381.0  Schuyler County Bancshares Inc    NaN  1100055.0    MO
7595   35274.0       Blue Ridge Bankshares Inc    NaN  1426960.0    VA
7860    8252.0        Jamesmark Bancshares Inc    NaN  2816803.0    MO
8189   20776.0           Border Bancshares Inc    NaN  2238544.0    MN
8648   29617.0         First Mutual Holding Co    NaN  4843090.0    OH
8846   57497.0            Pathfinder Bcorp Inc    NaN  2596776.0    NY
9132   29617.0         First Mutual Holding Co    NaN  4843090.0    OH
9182   11310.0                   Fgh Bcorp Inc    NaN  2527024.0    IL
9712    8888.0                    Ccb Fin Corp    NaN  1053496.0    MO
9832   17399.0                  First Fin Corp    NaN  1067514.0    KS
10230  13381.0  Schuyler County Bancshares Inc    NaN  1100055.0    MO
10268  17636.0     First Belleville Bancshares    NaN  1060917.0    KS
10459   1756.0     Cross County Bancshares Inc    NaN  1131611.0    AR
10547  57197.0                     Bnccorp Inc   BNCC  1248162.0    ND

In [62]:
bhcc[bhcc['xCert']==17399]

xCert          Parent Ticker    ParRSSD ParST
6767  17399.0  First Fin Corp    NaN  1067514.0    KS
9832  17399.0  First Fin Corp    NaN  1067514.0    KS

In [63]:
len(bhcc) # 4100

4100

In [64]:
bhcc2 = bhcc.drop_duplicates(subset=['xCert','Parent','Ticker','ParRSSD'],keep=False)
len(bhcc2) # 4058

4058

In [65]:
bhcc2.head()

xCert                Parent Ticker    ParRSSD ParST
0   628.0     Jpmorgan Chase&Co    JPM  1039502.0    NY
1  3510.0  Bank Of America Corp    BAC  1073757.0    NC
2  7213.0         Citigroup Inc      C  1951350.0    NY
3  3511.0   Wells Fargo&Company    WFC  1120754.0    CA
5  6548.0             U S Bcorp    USB  1119794.0    MN

In [66]:
bhcc2.set_index(['xCert'],drop=True,inplace=True)
bhcc2.head()

Parent Ticker    ParRSSD ParST
xCert                                               
628.0      Jpmorgan Chase&Co    JPM  1039502.0    NY
3510.0  Bank Of America Corp    BAC  1073757.0    NC
7213.0         Citigroup Inc      C  1951350.0    NY
3511.0   Wells Fargo&Company    WFC  1120754.0    CA
6548.0             U S Bcorp    USB  1119794.0    MN

In [67]:
bhcc2.groupby(level=0).filter(lambda x: len(x) > 1)  ## check for duplicates in xCert index

Empty DataFrame
Columns: [Parent, Ticker, ParRSSD, ParST]
Index: []

In [68]:
dcrt = bhcc2[['Parent','Ticker','ParRSSD','ParST']].to_dict()
dcrt['Parent'][628]

'Jpmorgan Chase&Co'

In [69]:
##### LOOK AT THIS!!! IMPORTANT CONDITIONAL PROCESSING FOR MISSING VALUES!!!!  ## improved 5/9/2016
##########################################################################
j = 0
rk5c = rk5.copy()
for i in np.arange(len(rk5c)):
  if (pd.isnull(rk5c.loc[i]['Parent']))|(rk5c.loc[i]['Parent']==' '):
    xc = rk5c.loc[i]['UltCert']
    try:
      rk5c.loc[i,'Parent'] = dcrt['Parent'][xc]
      rk5c.loc[i,'Ticker'] = dcrt['Ticker'][xc]
      rk5c.loc[i,'ParRSSD'] = dcrt['ParRSSD'][xc]
      rk5c.loc[i,'ParST'] = dcrt['ParST'][xc]
      j = j+1
    except:
      pass

In [70]:
len(rk5c[rk5c['Ticker']>'']) # 3514

3514

In [71]:
j # 5071

5071

In [72]:
len(rk5c) # 10798

10798

In [73]:
rk5c.to_csv("BkAvgAstPR_FailDate_BHC2_ult202012.csv")

In [74]:
# I use ZION to show how many historic banks and mergers got collapsed into one bank charter

In [75]:
rk4.loc[rk4.Ticker.isin(['ZION']),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']] # current bank

IDRSSD                     xName   xCert Ticker Parent    ParRSSD
73  276579  ZIONS BANCORPORATION, NA  2270.0   ZION   ZION  1027004.0

In [76]:
rk5c.loc[rk5c.Ticker.isin(['ZION']),['IDRSSD','xName','xCert','Ticker','Parent','ParRSSD']] #historic banks merged in Zions 

IDRSSD                                 xName    xCert Ticker Parent  \
73     276579              ZIONS BANCORPORATION, NA   2270.0   ZION   ZION   
161    837260               CALIFORNIA BANK & TRUST  20852.0   ZION   ZION   
165    676656                        AMEGY BANK, NA  24107.0   ZION   ZION   
325   1004368              NATIONAL BANK OF ARIZONA  20626.0   ZION   ZION   
349    456960                     NEVADA STATE BANK  18113.0   ZION   ZION   
462    933957              VECTRA BANK COLORADO, NA   2993.0   ZION   ZION   
745    807665                     VINEYARD BANK, NA  23556.0   ZION   ZION   
1165   458263                  STOCKMEN'S BANK, THE  23182.0   ZION   ZION   
1207  1198641  COMMERCE BANK OF WASHINGTON, NA, THE  27298.0   ZION   ZION   
1278  2479310                     SILVER STATE BANK  34194.0   ZION   ZION   
1731   720465                         ALLIANCE BANK  23124.0   ZION   ZION   
1796   618656                            KLEIN BANK  22866.0   ZION   ZION   
2833   105950          MINNEQUA BANK OF PUEBLO, THE   1773.0   ZION   ZION   
3329   576158                             MAXIMBANK   9017.0   ZION   ZION   
4256   309664                           RANCHO BANK  22934.0   ZION   ZION   
4349   521550                        LONE STAR BANK  22304.0   ZION   ZION   
4809  2068134            GREAT BASIN BANK OF NEVADA  33824.0   ZION   ZION   
6360   626754              LEAGUE CITY BANK & TRUST  17577.0   ZION   ZION   
6563  3161993                           CHOICE BANK  57469.0   ZION   ZION   
6842   687269        INTERCONTINENTAL NATIONAL BANK  19112.0   ZION   ZION   
6869   456670                   FRONTIER STATE BANK  24227.0   ZION   ZION   
8511  3374186          COMMERCE BANK OF OREGON, THE  58223.0   ZION   ZION   
9952  2907448               SOUTHLAND BUSINESS BANK  35287.0   ZION   ZION   

        ParRSSD  
73    1027004.0  
161   1027004.0  
165   1027004.0  
325   1027004.0  
349   1027004.0  
462   1027004.0  
745   1027004.0  
1165  1027004.0  
1207  1027004.0  
1278  1027004.0  
1731  1027004.0  
1796  1027004.0  
2833  1027004.0  
3329  1027004.0  
4256  1027004.0  
4349  1027004.0  
4809  1027004.0  
6360  1027004.0  
6563  1027004.0  
6842  1027004.0  
6869  1027004.0  
8511  1027004.0  
9952  1027004.0